In [40]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
import keras_tuner as kt

from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [41]:
# array normalization
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [42]:
data = load_wine()

In [43]:
list(data.target_names)

['class_0', 'class_1', 'class_2']

In [44]:
data['feature_names']

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [45]:
type(data)

sklearn.utils.Bunch

In [46]:
data['data'].shape

(178, 13)

In [47]:
data['target'].shape

(178,)

In [48]:
data1 = pd.DataFrame(data= np.c_[data['data'], data['target']],
                     columns= data['feature_names'] + ['target_names'])

In [49]:
data1['target_names'] = data1['target_names'].astype('int64')

In [50]:
data1

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target_names
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [51]:
# input data, normalization
x = data1.copy().astype(np.float64).drop("target_names", axis=1)
x = normalize(x.to_numpy())
x

array([[1.32644724e-02, 1.59397384e-03, 2.26512072e-03, ...,
        9.69434383e-04, 3.65402190e-03, 9.92738094e-01],
       [1.25128005e-02, 1.68733218e-03, 2.02859038e-03, ...,
        9.95336401e-04, 3.22299406e-03, 9.95336401e-01],
       [1.10630135e-02, 1.98394467e-03, 2.24454758e-03, ...,
        8.65874158e-04, 2.66487484e-03, 9.96175609e-01],
       ...,
       [1.57227449e-02, 5.07108879e-03, 2.67772446e-03, ...,
        6.99051960e-04, 1.84834078e-03, 9.89336248e-01],
       [1.55136606e-02, 3.05090212e-03, 2.79175213e-03, ...,
        7.06772691e-04, 1.90828627e-03, 9.89481768e-01],
       [2.48340486e-02, 7.20591644e-03, 4.81566123e-03, ...,
        1.07209976e-03, 2.81206495e-03, 9.84222734e-01]])

In [52]:
# formation of target data
columns = ['target_names']
y = pd.DataFrame(data1, columns=columns, dtype='int64')
y = y.to_numpy()
y = y.flatten()
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

In [53]:
# Separation of data into training and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [54]:
# training target
y_train

array([0, 2, 1, 1, 2, 0, 0, 0, 2, 0, 0, 1, 2, 1, 0, 2, 1, 0, 2, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 2, 2, 0, 1, 2, 2, 1, 1,
       0, 1, 2, 2, 1, 2, 1, 1, 1, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 2, 1, 1, 1, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0,
       1, 0, 2, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 1, 1, 1, 1,
       1, 1, 2, 0, 1, 1, 0, 1, 1], dtype=int64)

In [55]:
# training data
X_train

array([[1.31805705e-02, 1.78817368e-03, 2.26886553e-03, ...,
        1.06713590e-03, 3.84553479e-03, 9.95032127e-01],
       [3.27814817e-02, 1.19033622e-02, 5.26676544e-03, ...,
        1.36983137e-03, 3.14116504e-03, 9.80137963e-01],
       [2.81319418e-02, 3.22750512e-03, 4.53224124e-03, ...,
        2.81548319e-03, 6.27188939e-03, 9.79696591e-01],
       ...,
       [9.27438445e-03, 1.20605695e-03, 1.53498157e-03, ...,
        7.73940288e-04, 1.93485072e-03, 9.97738021e-01],
       [2.52769066e-02, 3.04757030e-03, 4.50163979e-03, ...,
        1.91220097e-03, 4.10326459e-03, 9.85978626e-01],
       [2.74502920e-02, 5.45001746e-03, 5.47226243e-03, ...,
        1.77959754e-03, 7.51879960e-03, 9.74329652e-01]])

In [56]:
X_train.shape

(119, 13)

In [57]:
# define the keras model
model = Sequential([
  Dense(256, activation='relu', input_shape=(13,)),
  Dense(256, activation='relu'),
  Dense(3, activation='softmax'),
])

In [58]:
# creating a keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [59]:
# education
# %%time
model.fit(X_train, 
          to_categorical(y_train), 
          epochs=45, 
          batch_size=32)

Epoch 1/45
4/4 [==============================] - 0s 4ms/step - loss: 1.1048 - accuracy: 0.3950
Epoch 2/45
4/4 [==============================] - 0s 1ms/step - loss: 1.0855 - accuracy: 0.3950
Epoch 3/45
4/4 [==============================] - 0s 1ms/step - loss: 1.0803 - accuracy: 0.3950
Epoch 4/45
4/4 [==============================] - 0s 2ms/step - loss: 1.0776 - accuracy: 0.3950
Epoch 5/45
4/4 [==============================] - 0s 1ms/step - loss: 1.0740 - accuracy: 0.3950
Epoch 6/45
4/4 [==============================] - 0s 1ms/step - loss: 1.0707 - accuracy: 0.3950
Epoch 7/45
4/4 [==============================] - 0s 2ms/step - loss: 1.0629 - accuracy: 0.3950
Epoch 8/45
4/4 [==============================] - 0s 1ms/step - loss: 1.0609 - accuracy: 0.4118
Epoch 9/45
4/4 [==============================] - 0s 1ms/step - loss: 1.0553 - accuracy: 0.5714
Epoch 10/45
4/4 [==============================] - 0s 2ms/step - loss: 1.0458 - accuracy: 0.5546
Epoch 11/45
4/4 [======================

In [60]:
# Evaluate the model.
model.evaluate(
  X_test,
  to_categorical(y_test)
)

2/2 [==============================] - 0s 0s/step - loss: 0.5938 - accuracy: 0.6780


[0.5938476324081421, 0.6779661178588867]

In [61]:
# Predict 
predictions = model.predict(X_test)

In [62]:
np.argmax(predictions, axis=1)

array([0, 0, 1, 0, 1, 0, 1, 2, 2, 0, 0, 1, 0, 1, 0, 1, 1, 2, 0, 1, 0, 1,
       2, 1, 1, 2, 1, 2, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2, 2, 0, 1, 1, 2,
       1, 0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 0, 1, 0, 1], dtype=int64)

In [63]:
print(classification_report(y_test, np.argmax(predictions, axis=1)))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93        20
           1       0.64      0.67      0.65        24
           2       0.36      0.27      0.31        15

    accuracy                           0.68        59
   macro avg       0.62      0.64      0.63        59
weighted avg       0.65      0.68      0.66        59



In [64]:
print(multilabel_confusion_matrix(y_test, np.argmax(predictions, axis=1)))

[[[36  3]
  [ 0 20]]

 [[26  9]
  [ 8 16]]

 [[37  7]
  [11  4]]]


---

In [65]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(13,)))

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-0, 1e-1, 1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [66]:
tuner = kt.Hyperband(build_model,
                     objective='accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir5',
                     project_name='intro_to_kt')

In [67]:
# %%time
tuner.search(X_train, y_train)

models = tuner.get_best_models(num_models=2)

tuner.results_summary()

Trial 30 Complete [00h 00m 00s]
accuracy: 0.6386554837226868

Best accuracy So Far: 0.6386554837226868
Total elapsed time: 00h 00m 22s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in my_dir5\intro_to_kt
Showing 10 best trials
Objective(name='accuracy', direction='max')
Trial summary
Hyperparameters:
units: 192
learning_rate: 0.1
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.6386554837226868
Trial summary
Hyperparameters:
units: 480
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: d175b6053b8e06a0b8f6412c336912b4
Score: 0.6302521228790283
Trial summary
Hyperparameters:
units: 288
learning_rate: 0.1
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 419e01e49b639dd6261f333ac6394632
Score: 0.5798319578170776
Trial summary
Hyperparameters:
units: 480
learning_rate: 0.01
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Scor

In [68]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 192 and the optimal learning rate for the optimizer
is 0.1.



In [70]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
3/3 [==============================] - 0s 49ms/step - loss: 1.7398 - accuracy: 0.3368 - val_loss: 1.2049 - val_accuracy: 0.2500
Epoch 2/50
3/3 [==============================] - 0s 6ms/step - loss: 1.1987 - accuracy: 0.3474 - val_loss: 1.1568 - val_accuracy: 0.6250
Epoch 3/50
3/3 [==============================] - 0s 6ms/step - loss: 1.1088 - accuracy: 0.4421 - val_loss: 1.0684 - val_accuracy: 0.4583
Epoch 4/50
3/3 [==============================] - 0s 7ms/step - loss: 1.0813 - accuracy: 0.3789 - val_loss: 1.0315 - val_accuracy: 0.4583
Epoch 5/50
3/3 [==============================] - 0s 6ms/step - loss: 1.0588 - accuracy: 0.3789 - val_loss: 1.0163 - val_accuracy: 0.4583
Epoch 6/50
3/3 [==============================] - 0s 6ms/step - loss: 1.0239 - accuracy: 0.3789 - val_loss: 0.9670 - val_accuracy: 0.5417
Epoch 7/50
3/3 [==============================] - 0s 6ms/step - loss: 0.9818 - accuracy: 0.5895 - val_loss: 0.9507 - val_accuracy: 0.6250
Epoch 8/50
3/3 [=================

In [71]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 15


In [72]:
hypermodel = tuner.hypermodel.build(best_hps)

In [73]:
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/15
3/3 [==============================] - 0s 59ms/step - loss: 1.5311 - accuracy: 0.3158 - val_loss: 1.0575 - val_accuracy: 0.4583
Epoch 2/15
3/3 [==============================] - 0s 7ms/step - loss: 1.1299 - accuracy: 0.3789 - val_loss: 1.0563 - val_accuracy: 0.4583
Epoch 3/15
3/3 [==============================] - 0s 7ms/step - loss: 1.1055 - accuracy: 0.3789 - val_loss: 1.0612 - val_accuracy: 0.4583
Epoch 4/15
3/3 [==============================] - 0s 7ms/step - loss: 1.0854 - accuracy: 0.3789 - val_loss: 1.0817 - val_accuracy: 0.4583
Epoch 5/15
3/3 [==============================] - 0s 6ms/step - loss: 1.0870 - accuracy: 0.4105 - val_loss: 1.0906 - val_accuracy: 0.4583
Epoch 6/15
3/3 [==============================] - 0s 7ms/step - loss: 1.0848 - accuracy: 0.4421 - val_loss: 1.0899 - val_accuracy: 0.4167
Epoch 7/15
3/3 [==============================] - 0s 6ms/step - loss: 1.0849 - accuracy: 0.4632 - val_loss: 1.0707 - val_accuracy: 0.5000
Epoch 8/15
3/3 [=================

In [75]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

2/2 [==============================] - 0s 999us/step - loss: 0.7843 - accuracy: 0.7458
[test loss, test accuracy]: [0.7842600345611572, 0.7457627058029175]


In [76]:
# Predict 
predictions = model.predict(X_test)

In [78]:
np.argmax(predictions, axis=1)

array([2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 0, 1, 0, 1, 0, 1, 1, 2, 0, 1, 0, 1,
       2, 1, 1, 2, 1, 2, 1, 0, 0, 1, 2, 0, 0, 0, 2, 1, 2, 2, 0, 1, 1, 2,
       1, 0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 1, 0, 1], dtype=int64)

In [79]:
print(classification_report(y_test, np.argmax(predictions, axis=1)))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95        20
           1       0.67      0.67      0.67        24
           2       0.40      0.40      0.40        15

    accuracy                           0.69        59
   macro avg       0.67      0.67      0.67        59
weighted avg       0.69      0.69      0.69        59



In [80]:
print(multilabel_confusion_matrix(y_test, np.argmax(predictions, axis=1)))

[[[38  1]
  [ 1 19]]

 [[27  8]
  [ 8 16]]

 [[35  9]
  [ 9  6]]]
